In [1]:
from skimpy import skim 
import json 
import requests 
import pandas as pd
from pandas import json_normalize 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_logs_vol = pd.read_csv('D:\\Top_gun\\Datasets\\df_logs_vols\\logs_vols_2024-06-10.csv', sep=',')

In [3]:
df_logs_vol.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V09972247,B757_0375,2024-06-06,3.5,"{'temp': '6.4°C', 'pressure': '911.2 hPa', 'vi...",0
1,V05194992,B747_2724,2024-06-06,5.4,"{'temp': '4.8°C', 'pressure': '1011.5 hPa', 'v...",0
2,V04977867,B747_0442,2024-06-06,3.6,"{'temp': '1.4°C', 'pressure': '788.9 hPa', 'vi...",0
3,V04836716,E170_6913,2024-06-06,4.6,"{'temp': '-27.4°C', 'pressure': '946.7 hPa', '...",1
4,V06062166,A330_2982,2024-06-06,2.2,"{'temp': '12.4°C', 'pressure': '983.1 hPa', 'v...",0


In [4]:
# Convertiion en dictionnaire python
def fix_json_format(x):
    x = x.replace('\'', '\"')
    return x
df_logs_vol['sensor_data'] = df_logs_vol['sensor_data'].apply(lambda x: json.loads(fix_json_format(x)))

# Normalisation de la colonne 'sensor_data'
sensor_data_df = pd.json_normalize(df_logs_vol['sensor_data'])

# Suppression de la colonne 'sensor_data' + concaténation des données normalisées
df_logs_vol = df_logs_vol.drop(columns=['sensor_data'])
df_logs_vols = pd.concat([df_logs_vol, sensor_data_df], axis=1)


df_logs_vols.head()

,ref_vol,aero_linked,jour_vol,time_en_air,etat_voyant,temp,pressure,vibrations
0,V09972247,B757_0375,2024-06-06,3.5,0,6.4°C,911.2 hPa,0.009413792309728164 m/s²
1,V05194992,B747_2724,2024-06-06,5.4,0,4.8°C,1011.5 hPa,0.16979882598674778 m/s²
2,V04977867,B747_0442,2024-06-06,3.6,0,1.4°C,788.9 hPa,0.17148269076934175 m/s²
3,V04836716,E170_6913,2024-06-06,4.6,1,-27.4°C,946.7 hPa,1.5543521695253382 m/s²
4,V06062166,A330_2982,2024-06-06,2.2,0,12.4°C,983.1 hPa,1.5906137497027713 m/s²


In [14]:
df_logs_vols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ref_vol             203 non-null    object        
 1   aero_linked         203 non-null    object        
 2   jour_vol            203 non-null    datetime64[ns]
 3   time_en_air         203 non-null    float64       
 4   etat_voyant         203 non-null    int64         
 5   temp en °C          203 non-null    float64       
 6   pressure en hPa     203 non-null    float64       
 7   vibrations en m/s²  203 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 12.8+ KB


In [12]:
# Fonction de nettoyage des données: 
def get_data_cleaning_report(df):
    df_logs_vols['jour_vol'] = pd.to_datetime(df_logs_vols['jour_vol'], format='%Y-%m-%d', errors='coerce')
    df_logs_vols['temp en °C'] = df_logs_vols['temp'].replace('°C', '', regex=True).astype(float)
    df_logs_vols['pressure en hPa'] = df_logs_vols['pressure'].replace('hPa', '', regex=True).astype(float)
    df_logs_vols['vibrations en m/s²'] = df_logs_vols['vibrations'].replace('m/s²', '', regex=True).astype(float).round(3)
    df_logs_vols.drop(columns=['temp', 'pressure', 'vibrations'], inplace=True)
    
get_data_cleaning_report(df_logs_vols)

In [13]:
df_logs_vols.head()

,ref_vol,aero_linked,jour_vol,time_en_air,etat_voyant,temp en °C,pressure en hPa,vibrations en m/s²
0,V09972247,B757_0375,2024-06-06,3.5,0,6.4,911.2,0.009
1,V05194992,B747_2724,2024-06-06,5.4,0,4.8,1011.5,0.170
2,V04977867,B747_0442,2024-06-06,3.6,0,1.4,788.9,0.171
3,V04836716,E170_6913,2024-06-06,4.6,1,-27.4,946.7,1.554
4,V06062166,A330_2982,2024-06-06,2.2,0,12.4,983.1,1.591


In [7]:
skim(df_logs_vols)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 203    │ │ string      │ 6     │                                                          │
│ │ Number of columns │ 8      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA   ┃ NA %   ┃ mean      ┃ sd        ┃ p0    ┃ p25    ┃ p50  ┃ p75    ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ time_en_air     │    0 │      0 │     5.083 │     2.636 │   0.6 │   3.25 │  4.9 │   6.65 │    13 │ ▅▇▇▃▂▁  │  │
│ │ etat_voyant     │    0 │      0 │    0.1626 │    0.3699 │     0 │      0 │    0 │      0 │     1 │ ▇    ▂  │  │
│ └─────────────────┴──────┴────────┴───────────┴───────────┴───────┴────────┴──────┴────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ref_vol                   │       0 │          0 │                            1 │                      203 │  │
│ │ aero_linked               │       0 │          0 │                            1 │                      203 │  │
│ │ jour_vol                  │       0 │          0 │                            1 │                      203 │  │
│ │ temp                      │       0 │          0 │                            1 │                      203 │  │
│ │ pressure                  │       0 │          0 │                            2 │                      406 │  │
│ │ vibrations                │       0 │          0 │                            2 │                      406 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [15]:
export_csv = df_logs_vols.to_csv('D:\\Top_gun\\Data_clean\\logs_vols_full_cleaned.csv', sep=',', index=None, header=True)